# Google Calendar Extractor

In [33]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [34]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]
TOKEN_PATH = "./../cfg/token.json"
CREDENTIALS_PATH = "./../cfg/credentials.json"

In [35]:
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists(TOKEN_PATH):
    creds = Credentials.from_authorized_user_file(TOKEN_PATH, SCOPES)

# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            CREDENTIALS_PATH, SCOPES
        )
        creds = flow.run_local_server(port=0)

# Save the credentials for the next run
with open(TOKEN_PATH, "w") as token:
    token.write(creds.to_json())

In [36]:
service = build("calendar", "v3", credentials=creds)

In [37]:
now = datetime.datetime.now(datetime.UTC).isoformat()[:-6] + "Z"  # 'Z' indicates UTC time
now

'2024-01-10T23:19:33.480077Z'

In [38]:
events_result = (
    service.events()
    .list(
        calendarId="primary",
        timeMin=now,
        maxResults=10,
        singleEvents=True,
        orderBy="startTime",
    )
    .execute()
)

In [39]:
events = events_result.get("items", [])

if not events:
    print("No upcoming events found.")

# Prints the start and name of the next 10 events
for event in events:
    start = event["start"].get("dateTime", event["start"].get("date"))
    print(start, event["summary"])

2024-01-11T09:00:00Z work
2024-01-11T17:30:00Z Have a meeting with JAC UK
2024-01-12T09:00:00Z work
2024-01-15T09:00:00Z work
2024-01-16T09:00:00Z work
2024-01-17T09:00:00Z work
2024-01-18T09:00:00Z work
2024-01-19T09:00:00Z work
2024-01-22T09:00:00Z work
2024-01-23T09:00:00Z work
